# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686497


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:04<00:42,  1.56s/it]

Rendering models:  13%|█▎        | 4/30 [00:04<00:30,  1.16s/it]

Rendering models:  17%|█▋        | 5/30 [00:05<00:22,  1.11it/s]

Rendering models:  20%|██        | 6/30 [00:05<00:19,  1.26it/s]

...

Rendering models:  83%|████████▎ | 25/30 [00:12<00:01,  3.40it/s]

Rendering models:  87%|████████▋ | 26/30 [00:12<00:01,  3.62it/s]

Rendering models:  93%|█████████▎| 28/30 [00:12<00:00,  4.36it/s]

Rendering models: 100%|██████████| 30/30 [00:13<00:00,  5.10it/s]

not-logged-in-20a275b53abec8db9f41    0.014688
not-logged-in-6cf01ec1c1cb535db647    0.000505
RockyGeo                              0.000099
not-logged-in-f6fac0763296d5759c01    0.000103
cschwefl                              0.000098
not-logged-in-76533fde9e5cc1f18955    0.000158
asucich                               0.000103
AdamMorse                             0.000118
not-logged-in-222c88bfb35ff39b20ba    0.004982
DarioB                                0.000087
41Letters                             0.000103
alexmnelson18                         0.000088
sallymcshane                          0.058429
Lavadude                              0.000196
not-logged-in-ee31e7d4412be094dae3    0.039011
hithereeveryone                       0.000282
Bambura                               0.016171
not-logged-in-f2ec051b0985517e83db    0.000207
Fiona77h                              0.000381
rstafford90                           0.000532
h4ranek                               0.000190
FR4N1O       

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())